In [3]:
import requests,bs4,string,csv,logging,traceback,os
from datetime import datetime
def scrape_financial_times(refresh):
    logging.basicConfig(filename="logs/scrape_financial_times.log", 
                    format='%(asctime)s %(message)s', 
                    filemode='a') 
    logger=logging.getLogger()
    logger.setLevel(logging.INFO)
    logger.info('Started Scrapping the data')
    if refresh.upper()=='Y':
        data_file='NewsData/financial_times.csv'
        condition=1==1
        os.remove(data_file)
    else:
        arch_data_file='NewsData/financial_times.csv'
        data_file='NewsData/financial_times_current.csv'
        condition=pageno<2
    pageno=1
    csv.register_dialect('myDialect',
    delimiter = '|',
    lineterminator = '\n',
    quoting=csv.QUOTE_NONE,
    skipinitialspace=True)
    while condition:
        if pageno==1:
            weblink='https://www.ft.com/oil'
        else:
            weblink='https://www.ft.com/oil?page='+str(pageno)
        fhand=requests.get(weblink)
        if (fhand.text=='Maximum page number exceeded'):
            break
        soup=bs4.BeautifulSoup(fhand.text,'html.parser')
        li_lst=soup.findAll('li',{'class':'o-teaser-collection__item o-grid-row'})
        with open(data_file, 'a') as f:
#             logger.info('Opened file '+data_file)
            writer = csv.writer(f, dialect='myDialect')
            for li in li_lst:
                try:
                    if li:
                        date_object = datetime.strptime(li.find('time',{'data-o-component':'o-date'}).text, "%A, %d %B, %Y")
                        writer.writerow([date_object.strftime("%d/%m/%Y"),li.find('div',{'class':'o-teaser__heading'}).text])
                except AttributeError as ae:
                    None
                except UnicodeEncodeError as ue:
                    logger.error('Error While getting data from '+weblink+'ERROR:['+traceback.format_exc()+']')
        f.close()
        fhand.close()
        logger.info('Scrapping Completed for '+weblink)
        pageno=pageno+1
    
    if refresh.upper()=='N':
        readFile = open(data_file, "r")
        current_news = fin.read()
        readFile.close()
        logger.info('Data Read From '+data_file)
        writeFile = open(arch_data_file, "a")
        writeFile.write(current_news)
        writeFile.close()
        logger.info('Data Appended to '+arch_data_file)
    print('completed Successfully')

In [4]:
scrape_financial_times('Y')

completed Successfully
